In [7]:
import requests
import time
import os
import csv

In [8]:
url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/BaselineResults_8cpu.csv'

In [9]:
def my_decorator(func):
    def calculate_execution_time(func, *args, **kwargs):
        start_time = time.time()
        result,_ = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = [
            ['Payload', 'Response', 'Time'],
        [result, _.status_code, f'{execution_time:.6f} seconds'],
    ]
    save_report(report_path, data)
    print(f"Execution time of {func.__name__}: {execution_time:.6f} seconds")
    return result, _

In [10]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            yield line.strip()

# Using the generator function to read lines
payloads = read_payloads(request_payload_file)   

In [11]:
def save_report(file_path, data):
    file_exists = os.path.exists(file_path)
    with open(file_path, 'a', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        if not file_exists:
            csv_writer.writerow(data[0])
        for row in data[1:]:
            csv_writer.writerow(row)

In [13]:
##Inference Function

@calculate_execution_time
def inference(payload_data=None):
    payload = payload_data
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer {0}'.format(lltoken)
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    return payload_data, response

Execution time of inference: 0.176516 seconds


In [14]:
#Model Inference(Request)

for line_number, line_content in enumerate(payloads, start=0):
    print(f"Line {line_number}: {line_content}") 
    try:
        a,b = inference(line_content)
    except Exception as e:
        print(e)
    

Line 0: {"payload":"Problem: Write a function in python to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n."}
'tuple' object is not callable
Line 1: {"payload":"Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s."}
'tuple' object is not callable
Line 2: {"payload":"Problem: Write a function in python to check if a given number is a prime number.Input: Positive integer num.Output: Boolean, indicating whether num is prime."}
'tuple' object is not callable
Line 3: {"payload":"Problem: Write a function in python to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome."}
'tuple' object is not callable
Line 4: {"payload":"Problem: Write a function in python to calculate the sum of the digits of a given positive integer.Input: Positive integer num.Output: Integer, the sum of the digits of num."}